In [ ]:
from authentic_generator_np import DemandGenerator
import numpy as np

Parameters
--------------------
| Parameter        | Type          | Description                                                                                                   | Default              |
|-----------------|--------------|---------------------------------------------------------------------------------------------------------------|----------------------|
| `P`             | `int`        | Total number of ports.                                                                                        | **required**         |
| `C`             | `int`        | Total vessel capacity (number of containers that can be carried).                                             | **required**         |
| `target_utils`  | `list[float]`| Target utilization fractions per loading port. The length of target_utils should match the number of loading ports (P//2 if loading_only=True, otherwise P-1).                                                                | **required**         |
| `middle_leg`    | `int`        | Index separating loading and discharging ports.                                                               | `P // 2` if `None`   |
| `loading_only`  | `bool`       | If `True`, loading ports only ship to discharging ports.                                                      | `False`              |
| `sparsity`      | `float`      | Probability of zeroing an OD pair in the matrix, in `[0, 1]`.                                                 | `0.3`                |
| `perturb`       | `float`      | Fractional perturbation applied to matrix entries, in `[0, 1]`. Amount of random variation applied to each entry in the generated demand matrix.                                               | `0.2`                |
| `cargo_shares`  | `list`       | Shares for each cargo type (will be normalized). If `None`, uniform shares are used.                         | `None`               |
| `include_reefer`| `bool`       | Whether to include reefer cargo types.                                                                        | `True`               |
| `distribution`  | `str`        | Distribution used for randomization (`"poisson"`, `"neg_binomial"`, `"lognormal"`, `"normal"`, `"uniform"`).  | `"poisson"`          |
| `cv_demand`     | `float`      | Coefficient of variation of distribution. Common values `{0.5, 1.0, 1.5}`, but in `(0, ∞)`. It controls the relative amount of randomness (variation) in the generated demand scenarios.                  | `1.0`                |
| `seed`          | `int`        | Random seed for reproducibility.                                                                              | `None`               |


# Stochastic

## Single generation

In [ ]:
### Generate LD for the current port
P = 7
C = 7476
target_utils = np.ones(1)  # Length should match number of loading ports
current_port = 1
middle_leg = None
loading_only = False

seed = 98364998

n_scenarios = 5     # Number of scenarios

for n_scenarios in 

dg = DemandGenerator(
    P=P,
    C=C,
    target_utils=target_utils,
    current_port=current_port,
    include_current_port=True,
    middle_leg=middle_leg,
    loading_only=loading_only,
    sparsity=0.0,
    perturb=0.1,
    distribution="normal",
    seed=seed
)

loading_list = dg.generate_loading_list()

### Generate demand scenarios for all sucessive ports
n_scenarios = 1     # Number of scenarios

# Example target_utils for loading_only=True (length = P//2)
target_utils = np.ones(P-1) * 0.95 # Length 6 for P=7 and loading_only=False
include_current_port = False
current_port_ld = loading_list

# Generate data for ports from start_port to start_port + P - 1
dg = DemandGenerator(
    P = P,
    C = C,
    target_utils = target_utils,
    current_port = current_port,
    include_current_port = False,
    current_port_ld = current_port_ld,
    middle_leg = None,
    loading_only = loading_only,
    sparsity = 0.0,
    perturb = 0.3,
    cargo_shares = None,
    distribution = "normal",
    cv_demand = 1.0,
    seed = seed
)

expected_demand, std_demand = dg._generate_moments()
scenarios = dg._generate(expected_demand, std_demand, n_scenarios=n_scenarios)

In [ ]:
# Analyze onboard demand and utilization for the LD generated for the current port
ob_demand_ld = []
ob_teus_ld = []
def onboard_transports(ports, pol):
    return [(i, j) for i in range(ports) for j in range(ports) if i <= pol and j > pol]
for pol in range(P - 1):
    ob = 0
    ob_teu = 0
    k = 0
    for key in dg.cargo_types:
        for (oi, dj) in onboard_transports(P, pol):
            ob += loading_list[key][oi, dj]
            ob_teu += loading_list[key][oi, dj] * dg.teu[k]
        k += 1
    ob_demand_ld.append(int(ob))
    ob_teus_ld.append(int(ob_teu))
print("LD onboard demand per port:", ob_demand_ld)
print("LD onboard TEU per port:", ob_teus_ld)
actual_utils_ld = np.array(ob_teus_ld) / C
print("LD actual utilizations:", actual_utils_ld)
print("-" * 40)
print("-" * 40)

# Analyze onboard demand and utilization for each scenario
for i, scenario in enumerate(scenarios):
    print(f"Scenario {i+1}:")
    ob_demand = []
    ob_teus = []
    def onboard_transports(ports, pol):
        return [(i, j) for i in range(ports) for j in range(ports) if i <= pol and j > pol]
    for pol in range(P - 1):
        ob = 0
        ob_teu = 0
        for (oi, dj) in onboard_transports(P, pol):
            k = 0
            for key in dg.cargo_types:
                ob += scenario[key][oi, dj]
                ob_teu += scenario[key][oi, dj] * dg.teu[k]
                k += 1
        ob_demand.append(int(ob))
        ob_teus.append(int(ob_teu))
    print(f"  Onboard demand per port: {ob_demand}")
    print(f"  Onboard TEU per port: {ob_teus}")
    actual_utils = np.array(ob_teus) / C
    print(f"  Actual utilizations: {actual_utils}")
    print("-" * 40)

In [ ]:
# Export loading_list to a .txt file
FileName_LD = f"S_S_{P}_{current_port}_{n_scenarios}_{loading_only}__{middle_leg}_{seed}CP.txt"

output_file_ld = FileName_LD
with open(output_file_ld, "w") as f:
    # Write first line: number of ports and 1 scenario (LD is deterministic)
    f.write(f"{P} 1\n")
    # Write container types (remove 'ft' from size)
    for ctype in dg.cargo_types:
        size, weight, ctype_str = ctype
        size_int = int(str(size).replace('ft', ''))
        f.write(f"{size_int} {weight} {ctype_str}\n")
    # Write the LD matrices
    for ctype in dg.cargo_types:
        matrix = loading_list[ctype]
        for row in matrix:
            f.write(" ".join(str(int(x)) for x in row) + "\n")
print(f"LD exported to {output_file_ld}")

# Data file name, automatically set based on number of ports and scenarios
FileName = f"S_{P}_{current_port}_{n_scenarios}_{loading_only}.txt"

# Export scenarios to a .txt file
output_file = FileName
with open(output_file, "w") as f:
    # Write first line: number of ports and number of scenarios
    f.write(f"{P} {n_scenarios}\n")
    # Write container types (remove 'ft' from size)
    for ctype in dg.cargo_types:
        size, weight, ctype_str = ctype
        size_int = int(str(size).replace('ft', ''))
        f.write(f"{size_int} {weight} {ctype_str}\n")
    # Write all scenario matrices
    for scenario in scenarios:
        for ctype in dg.cargo_types:
            matrix = scenario[ctype]
            for row in matrix:
                f.write(" ".join(str(int(x)) for x in row) + "\n")
print(f"Scenarios exported to {output_file}")



In [ ]:
[1,2,3,4]

## Mass generation

In [ ]:
### Generate LD for the current port
P = 10
C = 7476
target_utils = np.ones(1)  # Length should match number of loading ports
current_port = 1
middle_leg = None
loading_only = False

seed = 98364998

n_scenarios = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,105,110,115,120,125,130,135,140,145,150,155,160,165,170,175,180,185,190,195,200]     # Number of scenarios

dg = DemandGenerator(
    P=P,
    C=C,
    target_utils=target_utils,
    current_port=current_port,
    include_current_port=True,
    middle_leg=middle_leg,
    loading_only=loading_only,
    sparsity=0.0,
    perturb=0.1,
    distribution="normal",
    seed=seed
)

loading_list = dg.generate_loading_list()

# Analyze onboard demand and utilization for the LD generated for the current port
ob_demand_ld = []
ob_teus_ld = []
def onboard_transports(ports, pol):
    return [(i, j) for i in range(ports) for j in range(ports) if i <= pol and j > pol]
for pol in range(P - 1):
    ob = 0
    ob_teu = 0
    k = 0
    for key in dg.cargo_types:
        for (oi, dj) in onboard_transports(P, pol):
            ob += loading_list[key][oi, dj]
            ob_teu += loading_list[key][oi, dj] * dg.teu[k]
        k += 1
    ob_demand_ld.append(int(ob))
    ob_teus_ld.append(int(ob_teu))
print("LD onboard demand per port:", ob_demand_ld)
print("LD onboard TEU per port:", ob_teus_ld)
actual_utils_ld = np.array(ob_teus_ld) / C
print("LD actual utilizations:", actual_utils_ld)
print("-" * 40)
print("-" * 40)

# Export loading_list to a .txt file
FileName_LD = f"S_S_{P}_CP_{loading_only}_{middle_leg}_{seed}.txt"

output_file_ld = FileName_LD
with open(output_file_ld, "w") as f:
    # Write first line: number of ports and 1 scenario (LD is deterministic)
    f.write(f"{P} 1\n")
    # Write container types (remove 'ft' from size)
    for ctype in dg.cargo_types:
        size, weight, ctype_str = ctype
        size_int = int(str(size).replace('ft', ''))
        f.write(f"{size_int} {weight} {ctype_str}\n")
    # Write the LD matrices
    for ctype in dg.cargo_types:
        matrix = loading_list[ctype]
        for row in matrix:
            f.write(" ".join(str(int(x)) for x in row) + "\n")
print(f"LD exported to {output_file_ld}")


for n in n_scenarios:

    ### Generate demand scenarios for all sucessive ports

    # Example target_utils for loading_only=True (length = P//2)
    target_utils = np.ones(P-1) * 0.95 # Length 6 for P=7 and loading_only=False
    include_current_port = False
    current_port_ld = loading_list

    # Generate data for ports from start_port to start_port + P - 1
    dg = DemandGenerator(
        P = P,
        C = C,
        target_utils = target_utils,
        current_port = current_port,
        include_current_port = False,
        current_port_ld = current_port_ld,
        middle_leg = middle_leg,
        loading_only = loading_only,
        sparsity = 0.0,
        perturb = 0.3,
        cargo_shares = None,
        distribution = "normal",
        cv_demand = 1.0,
        seed = seed
    )

    expected_demand, std_demand = dg._generate_moments()
    scenarios = dg._generate(expected_demand, std_demand, n_scenarios=n)

    # Analyze onboard demand and utilization for each scenario
    for i, scenario in enumerate(scenarios):
        print(f"Scenario {i+1}:")
        ob_demand = []
        ob_teus = []
        def onboard_transports(ports, pol):
            return [(i, j) for i in range(ports) for j in range(ports) if i <= pol and j > pol]
        for pol in range(P - 1):
            ob = 0
            ob_teu = 0
            for (oi, dj) in onboard_transports(P, pol):
                k = 0
                for key in dg.cargo_types:
                    ob += scenario[key][oi, dj]
                    ob_teu += scenario[key][oi, dj] * dg.teu[k]
                    k += 1
            ob_demand.append(int(ob))
            ob_teus.append(int(ob_teu))
        print(f"  Onboard demand per port: {ob_demand}")
        print(f"  Onboard TEU per port: {ob_teus}")
        actual_utils = np.array(ob_teus) / C
        print(f"  Actual utilizations: {actual_utils}")
        print("-" * 40)

    # Data file name, automatically set based on number of ports and scenarios
    FileName = f"S_S_{P}_{n}_{loading_only}_{middle_leg}_{seed}.txt"

    # Export scenarios to a .txt file
    output_file = FileName
    with open(output_file, "w") as f:
        # Write first line: number of ports and number of scenarios
        f.write(f"{P} {n}\n")
        # Write container types (remove 'ft' from size)
        for ctype in dg.cargo_types:
            size, weight, ctype_str = ctype
            size_int = int(str(size).replace('ft', ''))
            f.write(f"{size_int} {weight} {ctype_str}\n")
        # Write all scenario matrices
        for scenario in scenarios:
            for ctype in dg.cargo_types:
                matrix = scenario[ctype]
                for row in matrix:
                    f.write(" ".join(str(int(x)) for x in row) + "\n")
    print(f"Scenarios exported to {output_file}")



# Deterministic

In [ ]:
seed = 42             # or None to be non-deterministic
rng = np.random.default_rng(seed)

# 8-digit integers: 10_000_000 .. 99_999_999 (upper bound exclusive)
vec = rng.integers(10_000_000, 100_000_000, size=90)
print(vec)

In [ ]:
fix_seed_Deterministic = [18032585, 79656044, 68911436, 49499059, 48971371, 87273812, 17735107, 72763122,
 28132258, 18475961, 57383108, 97806011, 76217713, 78502573, 74572954, 80745787,
 56190391, 21530226, 85577340, 50534734, 55031672, 43371822, 26429467, 93408848,
 80341071, 67947860, 46217292, 84048545, 59088620, 49907277, 50541359, 30451484,
 18292232, 59912630, 89910085, 15743553, 87246216, 84486805, 34908372, 66849795,
 24870617, 78227896, 73047066, 41907337, 16112800, 97362822, 50111873, 90380900,
 71012711, 80054514, 78390949, 27517483, 42751542, 52004890, 54801238, 13942338,
 59191255, 23886054, 76903831, 71474405, 93027505, 42997839, 97075875, 46976531,
 39324282, 91498218, 43341373, 16870886, 81611197, 27052422, 51664651, 21692935,
 71783038, 52813443, 30421841, 60804990, 70283259, 94627548, 49343672, 24461934,
 84941037, 66678787, 73023859, 18750397, 38112997, 79105952, 84903382, 49169500,
 82428792, 85752268]

In [ ]:
from authentic_generator_np import DemandGenerator
import numpy as np

### Generate LD for the current port
P = 2
C = 17385
current_port = 1
loading_only = False

for q in fix_seed_Deterministic:
    if P == 11:
        P =2
    middle_leg = None #int(np.ceil(P/2))  # set this to "None" loadning_only = False.
    
    seed = q
    target_utils = np.ones(P-1)  # Length should match number of loading ports
    # Generate data for ports from start_port to start_port + P - 1
    dg = DemandGenerator(
        P = P,
        C = C,
        target_utils = target_utils,
        current_port = current_port,
        include_current_port = True,
        current_port_ld = None,
        middle_leg = middle_leg,
        loading_only = loading_only,
        sparsity = 0.0,
        perturb = 0.3,
        cargo_shares = None,
        distribution = "normal",
        cv_demand = 1.0,
        seed = seed
    )

    expected_demand, std_demand = dg._generate_moments()
    LD_Deterministic = dg._generate(expected_demand, std_demand, n_scenarios=1)

    # LD analysis for deterministic loading_list (safe indexing + handle list-of-scenarios)
    ob_demand_ld = []
    ob_teus_ld = []

    def onboard_transports(ports, pol):
        return [(i, j) for i in range(ports) for j in range(ports) if i <= pol and j > pol]

    # Normalize LD_Deterministic to a single scenario dict if it is a list
    if isinstance(LD_Deterministic, list):
        if len(LD_Deterministic) == 0:
            raise ValueError("LD_Deterministic is an empty list")
        LD_dict = LD_Deterministic[0]
    else:
        LD_dict = LD_Deterministic

    for pol in range(P - 1):
        ob = 0
        ob_teu = 0
        k = 0
        for key in dg.cargo_types:
            # now LD_dict should be a dict mapping cargo keys -> matrices
            matrix = LD_dict[key]
            rows, cols = matrix.shape
            for (oi, dj) in onboard_transports(P, pol):
                # skip if out-of-bounds for this cargo type
                if oi < rows and dj < cols:
                    val = matrix[oi, dj]
                    ob += val
                    ob_teu += val * dg.teu[k]
            k += 1
        ob_demand_ld.append(int(ob))
        ob_teus_ld.append(int(ob_teu))

    print("LD onboard demand per port:", ob_demand_ld)
    print("LD onboard TEU per port:", ob_teus_ld)
    actual_utils_ld = np.array(ob_teus_ld) / C
    print("LD actual utilizations:", actual_utils_ld)
    print("-" * 40)
    print("-" * 40)

    # Export LD_Deterministic to a .txt file
    FileName_LD = f"L_D_{P}_{loading_only}_{middle_leg}_{q}.txt"

    output_file_ld = FileName_LD
    # Normalize: if LD_Deterministic is a list of scenarios, take the first scenario (deterministic)
    if isinstance(LD_Deterministic, list):
        if len(LD_Deterministic) == 0:
            raise ValueError("LD_Deterministic is an empty list")
        LD_to_write = LD_Deterministic[0]
    else:
        LD_to_write = LD_Deterministic

    with open(output_file_ld, "w") as f:
        # Write first line: number of ports and 1 scenario (LD is deterministic)
        f.write(f"{P} 1\n")
        # Write container types (remove 'ft' from size)
        for ctype in dg.cargo_types:
            size, weight, ctype_str = ctype
            size_int = int(str(size).replace('ft', ''))
            f.write(f"{size_int} {weight} {ctype_str}\n")
        # Write the LD matrices (index by the same key used for loading_list)
        for ctype in dg.cargo_types:
            # use the cargo-type key the same way you used loading_list[ctype]
            try:
                matrix = LD_to_write[ctype]
            except Exception as e:
                # If that fails, try using the cargo-type string as key (third element)
                _, _, ctype_str = ctype
                if ctype_str in LD_to_write:
                    matrix = LD_to_write[ctype_str]
                else:
                    raise KeyError(f"Could not find LD matrix for cargo type {ctype!r}") from e

            # Write each row (cast to int like loading_list export)
            for row in matrix:
                f.write(" ".join(str(int(x)) for x in row) + "\n")

    print(f"LD exported to {output_file_ld}")
    P += 1


# Example use

In [ ]:
# ---------------- Example usage ----------------
if __name__ == "__main__":
    P = 6
    C = 20000
    loading_only = True
    middle_leg = P // 2

    # Make sure the lengths match the voyage length P
    if loading_only:
        target_utils = [0.6, 0.8, 1.0] # length = P//2
    else:
        target_utils = [0.6, 0.8, 1.0, 0.8, 0.6]  # length = P-1

    # slight randomization to target utils (example)
    target_utils = np.array(target_utils) * np.random.uniform(0.9, 1.1, size=len(target_utils))

    dg = DemandGenerator(
        P=P,
        C=C,
        target_utils=target_utils,
        middle_leg=middle_leg,
        loading_only=loading_only,
        sparsity=0.25,
        perturb=0.15,
    #   include_reefer=True,
        distribution="poisson",
        seed=42
    )

    expected_demand, std_demand = dg._generate_moments()
    demand = dg._generate(expected_demand, std_demand)
    cargo_sample = list(expected_demand.keys())[0]
    print("--------------------")
    print(f"Scenario 0 – Cargo type: {cargo_sample}")
    print(f"OD matrix shape: {demand[0][cargo_sample].shape}")
    print("OD matrix:\n", demand[0][cargo_sample])
    print(f"Total containers for this cargo type: {demand[0][cargo_sample].sum()}")

    # Analyze target vs actual utilizations
    def onboard_transports(ports: int, pol: int) -> np.array:
        """List of cargo groups that are onboard after port `pol`"""
        on_board = [(i, j) for i in range(ports) for j in range(ports) if i <= pol and j > pol]
        return np.array(on_board)

    ob_demand = []
    ob_teus = []
    transport_indices = [(i, j) for i in range(P) for j in range(P) if i < j]
    for pol in range(P - 1):
        ob = 0
        ob_teu = 0
        for (i, j) in onboard_transports(P, pol):
            k = 0
            for key in dg.cargo_types:
                ob += demand[0][key][i, j]
                ob_teu += demand[0][key][i, j] * dg.teu[k]
                k += 1
        ob_demand.append(ob)
        ob_teus.append(ob_teu)

    print("--------------------")
    print(f"Onboard demand per port: {ob_demand}")
    print(f"Onboard TEU per port: {ob_teus}")
    actual_utils = np.array(ob_teus) / C
    print("Target utilizations:", target_utils)
    print(f"Actual utilizations: {actual_utils}")
